In [1]:
from google.colab import files
uploaded = files.upload()

Saving orders.csv to orders.csv


In [2]:
from google.colab import files
uploaded = files.upload()

Saving users.json to users.json


In [3]:
from google.colab import files
uploaded = files.upload()

Saving restaurants.sql to restaurants.sql


In [4]:
import os
os.listdir('/content/')

['.config', 'orders.csv', 'users.json', 'restaurants.sql', 'sample_data']

In [5]:
import pandas as pd
import sqlite3

In [6]:
orders = pd.read_csv('/content/orders.csv')
orders.head()

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name
0,1,2508,450,18-02-2023,842.97,New Foods Chinese
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian


In [7]:
users = pd.read_json('/content/users.json')
users.head()

,user_id,name,city,membership
0,1,User_1,Chennai,Regular
1,2,User_2,Pune,Gold
2,3,User_3,Bangalore,Gold
3,4,User_4,Bangalore,Regular
4,5,User_5,Pune,Gold


In [8]:
conn = sqlite3.connect(':memory:')

with open('/content/restaurants.sql', 'r') as file:
    sql_script = file.read()

conn.executescript(sql_script)

In [9]:
restaurants = pd.read_sql("SELECT * FROM restaurants", conn)
restaurants.head()

,restaurant_id,restaurant_name,cuisine,rating
0,1,Restaurant_1,Chinese,4.8
1,2,Restaurant_2,Indian,4.1
2,3,Restaurant_3,Mexican,4.3
3,4,Restaurant_4,Chinese,4.1
4,5,Restaurant_5,Chinese,4.8


In [10]:
orders_users = pd.merge(
    orders,
    users,
    on='user_id',
    how='left'
)


In [11]:
final_df = pd.merge(
    orders_users,
    restaurants,
    on='restaurant_id',
    how='left'
)

In [12]:
final_df.head()

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_x,name,city,membership,restaurant_name_y,cuisine,rating
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0


In [13]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   order_id           10000 non-null  int64  
 1   user_id            10000 non-null  int64  
 2   restaurant_id      10000 non-null  int64  
 3   order_date         10000 non-null  object 
 4   total_amount       10000 non-null  float64
 5   restaurant_name_x  10000 non-null  object 
 6   name               10000 non-null  object 
 7   city               10000 non-null  object 
 8   membership         10000 non-null  object 
 9   restaurant_name_y  10000 non-null  object 
 10  cuisine            10000 non-null  object 
 11  rating             10000 non-null  float64
dtypes: float64(2), int64(3), object(7)
memory usage: 937.6+ KB


In [14]:
final_df.to_csv('final_food_delivery_dataset.csv', index=False)

In [15]:
files.download('final_food_delivery_dataset.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [16]:
final_df[final_df['membership'] == 'Gold']['order_id'].count()

np.int64(4987)

In [17]:
round(final_df[final_df['city'] == 'Hyderabad']['total_amount'].sum())

1889367

In [18]:
final_df['user_id'].nunique()

2883

In [19]:
round(
    final_df[final_df['membership'] == 'Gold']['total_amount'].mean(),
    2
)

np.float64(797.15)

In [20]:
final_df[final_df['rating'] >= 4.5]['order_id'].count()

np.int64(3374)

In [21]:
top_city = (
    final_df[final_df['membership'] == 'Gold']
    .groupby('city')['total_amount']
    .sum()
    .idxmax()
)

final_df[
    (final_df['membership'] == 'Gold') &
    (final_df['city'] == top_city)
]['order_id'].count()


np.int64(1337)

In [22]:
final_df[final_df['membership'] == 'Gold'] \
    .groupby('city')['total_amount'] \
    .sum() \
    .sort_values(ascending=False)


,total_amount
city,
Chennai,1080909.79
Pune,1003012.32
Bangalore,994702.59
Hyderabad,896740.19


In [23]:
final_df.groupby('cuisine')['total_amount'] \
    .mean() \
    .sort_values(ascending=False)


,total_amount
cuisine,
Mexican,808.021344
Italian,799.448578
Indian,798.466011
Chinese,798.389020


In [24]:
user_spend = final_df.groupby('user_id')['total_amount'].sum()
(user_spend > 1000).sum()


np.int64(2544)

In [25]:
bins = [3.0, 3.5, 4.0, 4.5, 5.0]
labels = ['3.0–3.5', '3.6–4.0', '4.1–4.5', '4.6–5.0']

final_df['rating_range'] = pd.cut(final_df['rating'], bins=bins, labels=labels)

final_df.groupby('rating_range')['total_amount'] \
    .sum() \
    .sort_values(ascending=False)


/tmp/ipython-input-3208765811.py:6: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  final_df.groupby('rating_range')['total_amount'] \


,total_amount
rating_range,
4.6–5.0,2197030.75
4.1–4.5,1960326.26
3.0–3.5,1881754.57
3.6–4.0,1717494.41


In [26]:
final_df[final_df['membership'] == 'Gold'] \
    .groupby('city')['total_amount'] \
    .mean() \
    .sort_values(ascending=False)

,total_amount
city,
Chennai,808.459080
Hyderabad,806.421034
Bangalore,793.223756
Pune,781.162243


In [27]:
restaurant_count = final_df.groupby('cuisine')['restaurant_id'].nunique()
revenue = final_df.groupby('cuisine')['total_amount'].sum()

pd.concat([restaurant_count, revenue], axis=1)

,restaurant_id,total_amount
cuisine,,
Chinese,120,1930504.65
Indian,126,1971412.58
Italian,126,2024203.80
Mexican,128,2085503.09


In [28]:
(final_df['membership'] == 'Gold').mean() * 100

np.float64(49.87)

In [29]:
final_df.groupby(['membership', 'cuisine'])['total_amount'] \
    .sum() \
    .sort_values(ascending=False)

membership  cuisine
Regular     Mexican    1072943.30
            Italian    1018424.75
Gold        Mexican    1012559.79
            Italian    1005779.05
Regular     Indian      992100.27
Gold        Indian      979312.31
            Chinese     977713.74
Regular     Chinese     952790.91
Name: total_amount, dtype: float64

In [30]:
final_df['order_date'] = pd.to_datetime(final_df['order_date'])
final_df['quarter'] = final_df['order_date'].dt.to_period('Q')

final_df.groupby('quarter')['total_amount'] \
    .sum() \
    .sort_values(ascending=False)


/tmp/ipython-input-3411562792.py:1: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  final_df['order_date'] = pd.to_datetime(final_df['order_date'])


,total_amount
quarter,
2023Q3,2037385.10
2023Q4,2018263.66
2023Q1,1993425.14
2023Q2,1945348.72
2024Q1,17201.50
